# Analysis LSR vs Lexical Efficiency

We run a set of retrieval engines on both, lexical (I.e., BM25 weights) and learned sparse embeddings to calculate how much, on average the efficiency gap between learned sparse and lexical is.

In [1]:
import pandas as pd

df = pd.read_json("lsr-vs-lexical.jsonl.gz", lines=True)

In [2]:
df[["Retrieval", "lexical", "retrieval_per_query.runtime_wallclock", "nDCG@10"]].groupby(["Retrieval", "lexical"]).describe(percentiles=[0.5, .9, .99])

retrieval_per_query.runtime_wallclock  \
                                                              count   
Retrieval             lexical                                         
duckdb                False                                    60.0   
                      True                                      5.0   
kannolo               False                                    55.0   
naive-search          False                                    60.0   
pyserini-lsr          False                                    60.0   
                      True                                      5.0   
pyterrier-naive       False                                     5.0   
pyterrier-pisa        False                                     5.0   
pyterrier-splade      False                                    60.0   
                      True                                      5.0   
pyterrier-splade-pisa False                                    60.0   
                      True                                      5.0   
pytorch-naive         False                                    60.0   
                      True                                      5.0   
seismic               False                                    60.0   
                      True                                      5.0   

                                                                             \
                                     mean        std        min         50%   
Retrieval             lexical                                                 
duckdb                False     33.114083  47.859442   4.680000   12.112500   
                      True       7.009333   1.438391   5.120000    6.666667   
kannolo               False      0.938318   0.163283   0.600000    0.900000   
naive-search          False     24.105472  22.533124   3.360000   10.040000   
pyserini-lsr          False     21.766153   6.326525  12.333333   19.330000   
                      True     137.551667  52.565645  82.280000  127.200000   
pyterrier-naive       False     17.206667   1.399008  15.820000   16.660000   
pyterrier-pisa        False      0.616333   0.163148   0.400000    0.620000   
pyterrier-splade      False     29.824514  13.668734  15.320000   24.307083   
                      True      22.607167   3.149283  19.720000   21.340000   
pyterrier-splade-pisa False      8.763042  11.413337   0.875000    2.990000   
                      True       0.845500   0.175716   0.687500    0.780000   
pytorch-naive         False      1.533361   0.492851   0.580000    1.520833   
                      True       1.851833   0.781068   0.980000    1.840000   
seismic               False      1.101778   0.810446   0.220000    0.690000   
                      True       0.036333   0.009159   0.020000    0.040000   

                                                                  nDCG@10  \
                                      90%         99%         max   count   
Retrieval             lexical                                               
duckdb                False    102.136000  177.533767  185.583333    60.0   
                      True       8.468000    8.532800    8.540000     5.0   
kannolo               False      1.140000    1.337917    1.416667    55.0   
naive-search          False     58.130000   73.783008   74.580000    60.0   
pyserini-lsr          False     30.824000   39.545400   40.100000    60.0   
                      True     190.535333  219.681533  222.920000     5.0   
pyterrier-naive       False     18.736000   19.273600   19.333333     5.0   
pyterrier-pisa        False      0.776000    0.833600    0.840000     5.0   
pyterrier-splade      False     48.542000   71.379567   80.440000    60.0   
                      True      26.105500   26.816800   26.895833     5.0   
pyterrier-splade-pisa False     28.438000   41.235058   46.460000    60.0   
                      True       1.028000    1.128800    1.140000     5.0   
pytorch-naive         F

# Generate Evaluation DataFrame

The file `lsr-vs-lexical-evaluation.jsonl.gz` used below was created with

```
lsr-benchmark evaluate './*/*/seismic' './*/*/pyterrier-splade-pisa' './*/*/pyterrier-splade' -o lsr-vs-lexical-evaluation.jsonl.gz
```

The results are intermediate, and these steps here slightly enrich the file `lsr-vs-lexical.jsonl.gz` and remove datasets that do not yet have all approaches executed.

In [1]:
import pandas as pd
from lsr_benchmark.datasets import all_embeddings
from tira.rest_api_client import Client
import json

df = pd.read_json('../runs/lsr-vs-lexical-evaluation.jsonl.gz', lines=True)
df["Retrieval"] = df["approach"].apply(lambda i: i.split("/")[-1])
del df["approach"]
APPROACHES = ["pyterrier-splade", "pyterrier-splade-pisa", "seismic", "duckdb"]
tira = Client()
d_stats = {}

for d, d_stat in json.loads(open('../lsr_benchmark/datasets/overview.json').read()).items():
    d_stats[d] = d_stat['dataset_stats']

In [2]:
def normalize_index_runtime(i):
    f = (d_stats[i["tira-dataset-id"]]["docs_count"]/1000)
    if f < 5:
        return None
    return int(i['index.runtime_wallclock'].split(' ')[0]) / f

def normalize_query_runtime(i):
    return int(i['retrieval.runtime_wallclock'].split(' ')[0]) / d_stats[i["tira-dataset-id"]]["queries_count"]

df['index_1000.runtime_wallclock'] = df.apply(normalize_index_runtime, axis=1)
df['retrieval_per_query.runtime_wallclock'] = df.apply(normalize_query_runtime, axis=1)

In [3]:
complete_datasets = set()

print(f"df complete: {len(df)}")

for dataset in df["ir-dataset-id"].unique():
    complete = True
    for approach in APPROACHES:
        if approach in ('pyterrier-naive', 'pyterrier-pisa') and len(df[(df["ir-dataset-id"] == dataset) & (df["Retrieval"] == approach)]) > 0:
            continue

        available_embeddings = df[(df["ir-dataset-id"] == dataset) & (df["Retrieval"] == approach)]["embedding/model"].unique()
        for e in all_embeddings():
            if e not in available_embeddings:
                complete = False
    if complete:
        complete_datasets.add(dataset)

df = df[df["ir-dataset-id"].isin(complete_datasets)]
print(f"df filtered: {len(df)}")

df["lexical"] = df["embedding/model"].map(lambda i: i == "bm25")
df.to_json('lsr-vs-lexical.jsonl.gz', lines=True, orient="records")

df complete: 1525
df filtered: 1421


In [4]:
df["Retrieval"].unique()

array(['pyterrier-splade', 'naive-search', 'pytorch-naive',
       'pyterrier-splade-pisa', 'seismic', 'kannolo', 'duckdb',
       'pyterrier-naive', 'pyterrier-pisa', 'pyserini-lsr'], dtype=object)